In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
#import datetime
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from nets import nets_factory
import matplotlib.pyplot as plt

In [2]:
def one_hot(label):
    r = np.zeros(total_labels, dtype=int)
    r[label] = 1
    return r.tolist()

In [3]:
def batch_iter(data, batch_size, num_epochs, shuffle=False):
    data = np.array(data)
    data_size = len(data)
    # 每个epoch的num_batch
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    print("num_batches_per_epoch:",num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [4]:
data = pd.read_pickle('data/final_data.pkl')
index = data.columns.values
data_length = data.shape[0]

weather_type = set(data['Weather'])
weather_dict = dict(zip(weather_type,list(range(len(weather_type)))))

total_labels = len(weather_type)


data['Weather'].replace(pd.Series(weather_dict), inplace=True)

#data['Weather'] = data['Weather'].update(pd.Series(weather_dict))
data['Weather'] = data['Weather'].apply(one_hot)


In [5]:
data = data.sample(frac=1).reset_index(drop=True)

In [6]:
pd.Series(weather_dict)

Clear                    11
Cloudy                    3
Drizzle                  10
Drizzle,Fog               4
Mainly Clear              9
Moderate Rain             6
Moderate Rain,Drizzle     0
Moderate Rain,Fog         1
Mostly Cloudy             8
Rain                      5
Rain Showers              2
Rain,Drizzle              7
Rain,Fog                 12
dtype: int64

In [7]:
print(len(pd.Series(weather_dict)))

13


In [8]:
X = data[index[-1]].values
y = data[index[-2]].values
x_train, x_test, y_train, y_test = train_test_split(X, y)
x_train = x_train / 256.0
x_test  = x_test  / 256.0

In [9]:
batch_size = 40
num_epochs = 20

num_labels = len(weather_dict)

In [10]:
num_labels

13

In [11]:
# define 2 placeholders
x = tf.placeholder(tf.float32, [None, 224, 224,3], name="input_x")
y = tf.placeholder(tf.float32, [None, num_labels], name="input_y")

train_network_fn = nets_factory.get_network_fn('alexnet_v2', 
                                                 num_classes = num_labels, 
                                                 weight_decay = 0.0005,
                                                is_training = True,
                                               
                                                )

prediction, end_points = train_network_fn(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.00001
decay_steps = 5000
decay_rate = 0.95
    
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_steps=decay_steps, 
                                           decay_rate=decay_rate, staircase=True)
    #optimizer = tf.GradientDescent(learning_rate)
    #optimizer.minimize(...my loss..., global_step=global_step)


add_global = global_step.assign_add(1)
with tf.control_dependencies([add_global]):
    #train_op = opt.minimise(loss)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
result_list = [[],[],[], []] #[[iterater],[training_accuracy],[testing_accuracy], [loss]]
test_array = np.array(list(zip(x_test,y_test)))
test_array = test_array[:100]
x_t, y_t = zip(*test_array)


In [ ]:
tf.set_random_seed(1)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
     
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
    i = 0
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord = coord)
    for batch in batches:
        i = i + 1
        x_batch, y_batch = zip(*batch)
        sess.run(train_step, feed_dict={x:x_batch, y:y_batch})
        
        if i % 1 == 0:

            acc_train = sess.run(accuracy, feed_dict = {x:x_batch, y:y_batch})
            acc_test = sess.run(accuracy, feed_dict = {x:x_t, y:y_t})
            #ts = sess.run(global_step)
            #lr = sess.run(learning_rate)
            result_list[0].append(i)
            result_list[1].append(acc_train)
            result_list[2].append(acc_test)
            print("iterator " + str(i) + ", testing accuracy %.4f" % acc_test , 
                  ", training accuracy %.4f" % acc_train #, 
                  #", learning rate %f" % lr 
                 )

num_batches_per_epoch: 131


1. Define a net
2. Prediction
3. Loss
4. Optimizer
5. Accuracy

In [ ]:
plt.plot(result_list[0], result_list[1])
plt.plot(result_list[0], result_list[2])
plt.show()

In [ ]:
np.argmax(y_t,1)